<a href="https://colab.research.google.com/github/scarlettogrady/train_dataset/blob/main/Assignment_2_0.22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports**

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import difflib
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
import pandas as pd
from scipy import stats

# **Data Import**

In [17]:
!curl -LJO https://raw.githubusercontent.com/scarlettogrady/train_dataset/main/train.csv
src1 = 'train.csv'

!curl -LJO https://raw.githubusercontent.com/scarlettogrady/train_dataset/main/test.csv
src2 = 'test.csv'


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1375k  100 1375k    0     0  3406k      0 --:--:-- --:--:-- --:--:-- 3403k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  331k  100  331k    0     0  1047k      0 --:--:-- --:--:-- --:--:-- 1048k


# **Pre-processing**

In [35]:
import pandas as pd
from scipy import stats

#from sklearn.linear_model import LinearRegression
#from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

def load(csv_file):
  dataset_original = pd.read_csv(csv_file)
  df = pd.DataFrame(dataset_original)
  return df


df = load(src1)
df_test = load(src2)
#non_categorical_cols = df.iloc[:, 1:].select_dtypes(exclude=['object']).columns.tolist()
#df.loc[:, non_categorical_cols] = df.loc[:, numeric_columns].apply(pd.to_numeric, errors='coerce')
df= df.dropna()

def OutlierRemoval(dataframe, threshold=1.5):
  non_categorical_cols = dataframe.iloc[:, 1:].select_dtypes(exclude=['object']).columns.tolist()

  Q1 = dataframe[non_categorical_cols].quantile(0.25)
  Q3 = dataframe[non_categorical_cols].quantile(0.75)
  IQR = Q3 - Q1

  lowerBound = Q1 - threshold * IQR
  upperBound = Q3 + threshold * IQR

  # Identify outliers based on the IQR method
  outliers = ((dataframe[non_categorical_cols] < lowerBound) | (dataframe[non_categorical_cols] > upperBound)).any(axis=1)

  # removed incomplete data and outliers
  no_outliers = dataframe[~outliers]
  return no_outliers

df = OutlierRemoval2(df)

df = df.drop_duplicates()

def StatusIndexing(df):
  # Dot notation (if the column name doesn't contain spaces or special characters)
  #print(df_test.column_name.head(10))
  status_spelling = {
      'superheated': 'Superheated',
      'Superheated ': 'Superheated',
      'superheated ': 'Superheated',
      'subcooled': 'Subcooled',
      'Subcooled ': 'Subcooled',
      'subcooled ': 'Subcooled',
      'Subcool': 'Subcooled',
      'Subcoled': 'Subcooled',
      'Superheat': 'Superheated',
      'Saperheated': 'Superheated'
  }
  # Replace the incorrect spellings with the correct values
  df.loc[:, 'Status'] = df['Status'].replace(status_spelling)

  # Map 'Superheated' to 1 and 'Subcooled' to 0
  status_mapping= {
      'Superheated': 1,
      'Subcooled': 0
  }

  # Map 'Superheated' to 1 and 'Subcooled' to 0
  df.loc[:, 'Status'] = df['Status'].map(status_mapping)

  return df

df = StatusIndexing(df)
df_test = StatusIndexing(df_test)

Obstacle_Volume = df['Obstacle Height (m)'] * df['Obstacle Width (m)']* df['Obstacle Thickness (m)']
df.insert(9, 'Obstacle Volume (m^3)', Obstacle_Volume)
df = df.drop(columns = ['Obstacle Width (m)', 'Obstacle Height (m)','Obstacle Thickness (m)'])

Obstacle_Volume = df_test['Obstacle Height (m)'] * df_test['Obstacle Width (m)']* df_test['Obstacle Thickness (m)']
df_test.insert(9, 'Obstacle Volume (m^3)', Obstacle_Volume)
df_test = df_test.drop(columns = ['Obstacle Width (m)', 'Obstacle Height (m)','Obstacle Thickness (m)'])

def normalisation(df):
  scaler = MinMaxScaler()
  target = None
  if 'Target Pressure (bar)' in df.columns:
        target = df['Target Pressure (bar)']
        df = df.drop(columns=['Target Pressure (bar)'])
        target = target.reset_index(drop=True)
  new_df = df.drop(columns = ['ID', 'Status'])
  normalised_data = scaler.fit_transform(new_df)
  normalised_df = pd.DataFrame(normalised_data, columns=new_df.columns)
  df_reset = df.reset_index(drop=True)
  normalised_df.insert(0, 'ID', df_reset['ID'] )
  normalised_df.insert(11, 'Status', df_reset['Status'])
  if target is not None:
    normalised_df.insert(normalised_df.shape[1], 'Target Pressure (bar)', target)

  return normalised_df

df_train = normalisation(df)
df_test = normalisation(df_test)
df_train

df_train = df_train.drop(columns = ['Sensor ID', 'Liquid Boiling Temperature (K)', 'Liquid Critical Temperature (K)', 'Liquid Critical Pressure (bar)'])
df_test = df_test.drop(columns = ['Sensor ID', 'Liquid Boiling Temperature (K)', 'Liquid Critical Temperature (K)', 'Liquid Critical Pressure (bar)'])

# **Model 1 - Linear Regression**

In [32]:
def LinearRegressionTrain(df_trainset):
    features_train = df_trainset.drop(columns=['Target Pressure (bar)', 'ID'])
    target_train = df_trainset['Target Pressure (bar)']

    # Splitting the training data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(features_train, target_train, test_size=0.2, random_state=42)

    # Creating an instance of Linear Regression model
    model = LinearRegression()

    # Fitting the model to the training data
    model.fit(X_train, y_train)

    # Making predictions on the validation set
    y_pred = model.predict(X_val)

    # Evaluating the model
    mse = mean_squared_error(y_val, y_pred)
    print("Mean Squared Error on Validation Set:", mse)

    mape = mean_absolute_percentage_error(y_val, y_pred)
    print("Mean Absolute Percentage Error on Validation Set:", mape)

    return model

def LinearRegressionTest(test_dataset, LRModel):
    features_test = test_dataset.drop(columns=['ID'])  # Exclude 'ID' from features
    test_predictions = model.predict(features_test)

    # Creating a DataFrame for the predictions with 'ID' as the first column and predicted 'Target Pressure' as the second column
    df_test_predictions = pd.DataFrame({'ID': test_dataset['ID'], 'Predicted Target Pressure (bar)': test_predictions})
    # Saving the predictions to a CSV file
    df_test_predictions.to_csv('LR_test_predictions.csv', index=False)

    return df_test_predictions

**Testing Model 1**

In [33]:
"""df_train = preProcessingTrain(src1)

df_test = preProcessingTrain(src2)

df_train.iloc[:, 9:20]"""

model = LinearRegressionTrain(df_train)
df_test_predictions = LinearRegressionTest(df_test, model)

#df_test_predictions

Mean Squared Error on Validation Set: 0.04332851199591857
Mean Absolute Percentage Error on Validation Set: 0.8746545227200334


# **Model 2 - Random Forest Trees**

In [28]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

def RandomTreesModel(train_dataset, params = None):
    features_train = train_dataset.drop(columns=['Target Pressure (bar)', 'ID'])
    target_train = train_dataset['Target Pressure (bar)']

    # Splitting the training data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(features_train, target_train, test_size=0.2, random_state=42)

    if params is None:
        # Create the base model to tune
        rf_regressor = RandomForestRegressor(random_state=42)
        # Define the hyperparameter grid
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
        }

        # Random search of parameters, using 3 fold cross validation,
        # search across 100 different combinations, and use all available cores
        random_search = RandomizedSearchCV(estimator = rf_regressor, param_distributions = param_grid, n_iter = 50, cv = 3, verbose=2, random_state=42, n_jobs = -1)
        # Fit the random search model
        random_search.fit(X_train, y_train)

        print("Best RFT Hyperparameters:", random_search.best_params_)

        # Get the best model
        best_rf_regressor = random_search.best_estimator_

    else:
        # Use the provided hyperparameters
        best_rf_regressor = RandomForestRegressor(**params)


    best_rf_regressor.fit(X_train, y_train)

    # Make predictions on the validation set
    y_pred = best_rf_regressor.predict(X_val)

    # Calculate evaluation metrics
    mse = mean_squared_error(y_val, y_pred)
    print("Mean Squared Error on Validation Set:", mse)

    mape = mean_absolute_percentage_error(y_val, y_pred)
    print("Mean Absolute Percentage Error on Validation Set:", mape)

    return best_rf_regressor


def test_model(test_dataset, model):
    features_test = test_dataset.drop(columns=['ID'])

    # Make predictions on the test set
    y_pred = model.predict(features_test)

    # Creating a DataFrame for the predictions with 'ID' as the first column and predicted 'Target Pressure' as the second column
    df_test_predictions = pd.DataFrame({'ID': test_dataset['ID'], 'Predicted Target Pressure (bar)': y_pred})
    # Saving the predictions to a CSV file
    df_test_predictions.to_csv('test_predictions.csv', index=False)

    return df_test_predictions


**Testing Model 2**

In [30]:
# Assuming regressor is the trained Random Forest regressor model
#df_train = preProcessingTrain(src1)
#df_test = preProcessingTrain(src2)
best_params = {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 20}
#best_params = None
regressor = RandomTreesModel(df_train, best_params)
df_test_pred = test_model(df_test, regressor)
df_test_pred

Mean Squared Error on Validation Set: 0.0031413556665891565
Mean Absolute Percentage Error on Validation Set: 0.17794945454462496


,ID,Predicted Target Pressure (bar)
0,0,0.263057
1,1,0.267385
2,2,0.318678
3,3,0.311816
4,4,0.292456
...,...,...
3198,3198,0.592675
3199,3199,0.537284
3200,3200,0.407492
3201,3201,0.455029


# **Model 3 - Neural Networks**

In [21]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import random

def NeuralMLPNetwork(train_dataset, best_params=None, random_State=456):
    random.seed(random_State)  # Fixing the random seed for Python's random number generator
    np.random.seed(random_State)
    features_train = train_dataset.drop(columns=['Target Pressure (bar)', 'ID'])
    target_train = train_dataset['Target Pressure (bar)']

    # Splitting the training data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(features_train, target_train, test_size=0.2, random_state=random_State)

    if best_params is None:
        # If best_params is not provided, perform grid search to find the best hyperparameters
        mlp_regressor = MLPRegressor(random_state=random_State)

       ###param_grid = {
            #'hidden_layer_sizes': [(200, 150, 100, 50), (250, 200, 150, 100), (300, 200, 100)],
           # 'alpha': [0.0001, 0.001, 0.01],
           # 'learning_rate_init': [0.001, 0.01, 0.1]
        #}

        param_grid = {
            'hidden_layer_sizes': [(300, 200, 100), (250, 200, 150, 100), (400, 300, 200, 100)],  # Include the original configuration
            'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0],  # Adjusted range and granularity
            'learning_rate_init': [0.0001, 0.001, 0.01, 0.1, 1.0]  # Adjusted range and granularity
        }
        grid_search = GridSearchCV(mlp_regressor, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
        grid_search.fit(X_train, y_train)

        print("Best NN Hyperparameters:", grid_search.best_params_)
        best_model = grid_search.best_estimator_
    else:
        # Use the provided hyperparameters
        best_model = MLPRegressor(random_state = random_State, **best_params)

    best_model.fit(X_train, y_train)

    y_pred_val = best_model.predict(X_val)

    # Calculate Mean Absolute Error on Validation Set
    mae = mean_absolute_percentage_error(y_val, y_pred_val)
    print("Mean Absolute Percentage Error on Validation Set:", mae)

    return best_model

def NN_test_model(test_dataset, model, random_State=0):
    random.seed(random_State)  # Fixing the random seed for Python's random number generator
    np.random.seed(random_State)
    features_test = test_dataset.drop(columns=['ID'])

    # Make predictions on the test set
    y_pred_test = model.predict(features_test)

    # Creating a DataFrame for the predictions with 'ID' as the first column and predicted 'Target Pressure' as the second column
    df_test_predictions = pd.DataFrame({'ID': test_dataset['ID'], 'Predicted Target Pressure (bar)': y_pred_test})
    # Saving the predictions to a CSV file
    df_test_predictions.to_csv('NN_test_predictions.csv', index=False)

    return df_test_predictions


**Testing Model 3**

In [36]:
best_hyperparameters = {'alpha': 0.01, 'hidden_layer_sizes': (250, 200, 150, 100), 'learning_rate_init': 0.001}
#best_hyperparameters = None
NNMLP = NeuralMLPNetwork(df_train, best_params=best_hyperparameters)
#NNMLP = NeuralMLPNetwork(df_cleaned3)
df_NN_test_pred = NN_test_model(df_test, NNMLP)
df_NN_test_pred

Mean Absolute Percentage Error on Validation Set: 0.15976627582511102


,ID,Predicted Target Pressure (bar)
0,0,0.322594
1,1,0.349471
2,2,0.395597
3,3,0.289678
4,4,0.305450
...,...,...
3198,3198,0.582244
3199,3199,0.396259
3200,3200,0.306051
3201,3201,0.371475


**Ensemble Neural Network - to reduce variability**

In [57]:
def train_neural_network(train_dataset, random_seeds, best_params = None):
    models = []
    for seed in random_seeds:
        features_train = train_dataset.drop(columns=['Target Pressure (bar)', 'ID'])
        target_train = train_dataset['Target Pressure (bar)']

        # Splitting the training data into training and validation sets
        X_train, X_val, y_train, y_val = train_test_split(features_train, target_train, test_size=0.2, random_state=seed)

        if best_params is None:
          # If best_params is not provided, perform grid search to find the best hyperparameters
          mlp_regressor = MLPRegressor(random_state=seed)

          param_grid = {
              'hidden_layer_sizes': [(200, 150, 100, 50), (250, 200, 150, 100), (300, 200, 100)],
              'alpha': [0.0001, 0.001, 0.01],
              'learning_rate_init': [0.001, 0.01, 0.1]
          }
          grid_search = GridSearchCV(mlp_regressor, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
          grid_search.fit(X_train, y_train)

          print("Best NN Hyperparameters:", grid_search.best_params_)
          best_model = grid_search.best_estimator_
        else:
          # Use the provided hyperparameters
          best_model = MLPRegressor(random_state=seed, **best_params)

        best_model.fit(X_train, y_train)

        models.append(best_model)

        y_pred_val = best_model.predict(X_val)

        # Calculate Mean Absolute Error on Validation Set
        mae = mean_absolute_percentage_error(y_val, y_pred_val)
        print(f"For seed {seed}: Mean Absolute Percentage Error on Validation Set:", mae)

    return models

def ensemble_predict(test_dataset, models):
    features_test = test_dataset.drop(columns=['ID'])

    # Make predictions on the test set using each model
    all_predictions = np.array([model.predict(features_test) for model in models])

    # Average the predictions from all models
    averaged_predictions = np.mean(all_predictions, axis=0)

    df_test_predictions = pd.DataFrame({'ID': test_dataset['ID'], 'Predicted Target Pressure (bar)': averaged_predictions})
    # Saving the predictions to a CSV file
    df_test_predictions.to_csv('NN_ensemble_test_predictions.csv', index=False)

    return df_test_predictions


In [58]:
best_hyperparameters = {'alpha': 0.01, 'hidden_layer_sizes': (250, 200, 150, 100), 'learning_rate_init': 0.001}

random_seeds = [50, 123, 250, 300, 350, 400, 452, 456, 650, 1500]  # You can use different random seeds for different runs

trained_models = train_neural_network(df_train, random_seeds, best_hyperparameters)

aggregated_predictions = ensemble_predict(df_test, trained_models)
#NNMLP = NeuralMLPNetwork(df_cleaned3)
#df_NN_test_pred = NN_test_model(df_test, NNMLP)
aggregated_predictions

For seed 50: Mean Absolute Percentage Error on Validation Set: 0.17752976634775972
For seed 123: Mean Absolute Percentage Error on Validation Set: 0.154230943410197
For seed 250: Mean Absolute Percentage Error on Validation Set: 0.17444624430268627
For seed 300: Mean Absolute Percentage Error on Validation Set: 0.18649634261234077
For seed 350: Mean Absolute Percentage Error on Validation Set: 0.14715570937708325
For seed 400: Mean Absolute Percentage Error on Validation Set: 0.16241098254492933
For seed 452: Mean Absolute Percentage Error on Validation Set: 0.16588637243871293
For seed 456: Mean Absolute Percentage Error on Validation Set: 0.16390433496295845
For seed 650: Mean Absolute Percentage Error on Validation Set: 0.17618454181675702
For seed 1500: Mean Absolute Percentage Error on Validation Set: 0.21937797784213234


,ID,Predicted Target Pressure (bar)
0,0,0.281192
1,1,0.301924
2,2,0.322466
3,3,0.251099
4,4,0.266911
...,...,...
3198,3198,0.620284
3199,3199,0.448149
3200,3200,0.320385
3201,3201,0.388899
